In [53]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata('data.sfgov.org', None)

# Authenticate the client
domain = 'data.sfgov.org'
app_token = 'tZuHs7ko1xQezRBAgUN10D0i9'
username = 'corraljrmiguel@gmail.com'
password = 'Xd9@rzwPhZjKsNg8cima'

client = Socrata(domain,
                 app_token,
                 username=username,
                 password=password)

# Returns as JSON from API
# converted to Python list of dictionaries by sodapy.
results = client.get('vw6y-z8j6', limit=10000)
# results = client.get('vw6y-z8j6', where='service_request_id=12167455')

# Convert to pandas dataframe
df = pd.DataFrame.from_records(results)

In [60]:
df.shape

(10000, 47)

In [79]:
# # Pickle dataframe
# df.to_pickle('data/df_raw_10k.pkl')

# Load dataframe
df = pd.read_pickle('data/df_raw_10k.pkl')

In [80]:
# Drop all "computed_region" columns
df = df.loc[:, ~df.columns.str.contains('^:@')]
df.shape

(10000, 20)

In [81]:
def reorder_time_column(df):
    '''Reorder 'updated_datetime' column'''
    cols = df.columns.to_list()
    cols = cols[:3] + cols[-1:] + cols[3:-1]
    df = df[cols]
    return df

df = reorder_time_column(df)

- [ ] Find all duplicates
- [ ] Create column for duplicates
- [ ] Create base model
- [ ] Feature selection
- [ ] Feature engineering

In [178]:
df['is_duplicate'] = df['status_notes'].str.lower().str.contains('duplicate')

In [184]:
# Convert timestamps (strings) to date format
df.iloc[:, 1:4].to_timedelta()

,requested_datetime,updated_datetime,closed_date
0,2020-03-09T01:11:00.000,2020-03-09T01:13:08.000,NaN
1,2020-03-09T01:10:00.000,2020-03-09T03:34:13.000,NaN
2,2020-03-09T01:08:03.000,2020-03-09T01:08:04.000,NaN
3,2020-03-09T01:02:48.000,2020-03-09T01:02:51.000,NaN
4,2020-03-09T00:41:23.000,2020-03-09T02:10:01.000,NaN
...,...,...,...
9995,2020-03-03T14:51:00.000,2020-03-08T10:03:34.000,2020-03-08T10:03:34.000
9996,2020-03-03T14:51:00.000,2020-03-03T17:48:33.000,2020-03-03T17:48:33.000
9997,2020-03-03T14:50:40.000,2020-03-03T15:15:04.000,NaN
9998,2020-03-03T14:50:20.000,2020-03-03T16:00:54.000,2020-03-03T16:00:54.000
